# 04 - Simulation Production

Notebook de simulation du flow de production Chiron.

## Étapes
1. **Parsing PDF** : Extraction des données (pdfplumber ou Mistral OCR)
2. **Pseudonymisation** : (désactivée pour l'instant)
3. **Stockage** : (à venir)
4. **Génération LLM** : (à venir)

In [ ]:
# ruff: noqa: E402
import sys
from pathlib import Path

# Auto-détecter project_root
current = Path.cwd()
while current != current.parent:
    if (current / "pyproject.toml").exists():
        project_root = current
        break
    current = current.parent

sys.path.insert(0, str(project_root))

from dotenv import load_dotenv

load_dotenv(project_root / ".env")

# Paths
DATA_DIR = project_root / "data"
RAW_DIR = DATA_DIR / "raw"

print(f"Project root: {project_root}")
print(f"PDFs disponibles: {[p.name for p in RAW_DIR.glob('*.pdf')]}")

## 1. Configuration du Parser

Deux options disponibles :
- **pdfplumber** : Local, gratuit, rapide (~0.06s/page)
- **mistral_ocr** : Cloud, payant (2$/1000 pages), plus robuste (~2s/page)

Le choix se fait via :
- Variable d'environnement `PDF_PARSER_TYPE` dans `.env`
- Ou paramètre explicite `get_parser(ParserType.XXX)`

In [ ]:
from src.document import estimate_mistral_cost, get_parser
from src.llm.config import settings

# Afficher la config actuelle
print(f"Parser configuré dans .env : {settings.pdf_parser_type}")
print(f"Modèle Mistral OCR : {settings.mistral_ocr_model}")
print(f"Coût Mistral OCR : {settings.mistral_ocr_cost_per_1000_pages}$/1000 pages")

In [ ]:
# Estimation du coût si Mistral OCR
pdfs = list(RAW_DIR.glob("*.pdf"))
estimate = estimate_mistral_cost(pdfs)
print(f"PDFs à traiter : {len(pdfs)}")
print(f"Pages totales : {estimate['pages']}")
print(f"Coût estimé Mistral OCR : ${estimate['cost_usd']}")

In [ ]:
# Choisir le parser (décommenter celui souhaité)

# Option 1: Parser par défaut (depuis .env)
parser = get_parser()

# Option 2: Forcer pdfplumber (local, gratuit)
# parser = get_parser(ParserType.PDFPLUMBER)

# Option 3: Forcer Mistral OCR (cloud, payant)
# parser = get_parser(ParserType.MISTRAL_OCR)

print(f"Parser sélectionné : {type(parser).__name__}")

## 2. Parsing des PDFs

In [ ]:
import time

results = {}
times = {}

for pdf_path in sorted(RAW_DIR.glob("*.pdf")):
    eleve_id = pdf_path.stem

    start = time.perf_counter()
    try:
        extractions = parser.parse(pdf_path)
        results[eleve_id] = extractions[0] if extractions else None
    except Exception as e:
        results[eleve_id] = f"ERROR: {e}"
    times[eleve_id] = time.perf_counter() - start

    status = "OK" if not isinstance(results[eleve_id], str) else "ERREUR"
    print(f"{eleve_id}: {times[eleve_id]:.2f}s [{status}]")

print(f"\nTemps total: {sum(times.values()):.2f}s")
print(f"Temps moyen: {sum(times.values())/len(times):.2f}s/PDF")

In [ ]:
# Afficher un exemple de résultat
exemple_id = list(results.keys())[0]
exemple = results[exemple_id]

if exemple and not isinstance(exemple, str):
    print(f"=== {exemple_id} ===")
    print("raw_text (500 premiers chars):")
    print(exemple.raw_text[:500] if exemple.raw_text else "(vide)")
    print(f"\nraw_tables: {len(exemple.raw_tables or [])} table(s)")
else:
    print(f"Erreur: {exemple}")

## 3. Vérification des données extraites

Les parsers retournent des données brutes (`raw_text`, `raw_tables`).
Le post-traitement pour extraire les champs structurés sera fait par le LLM.

In [ ]:
# Résumé des extractions
import pandas as pd

summary = []
for eleve_id, extraction in results.items():
    if isinstance(extraction, str):
        summary.append({"eleve_id": eleve_id, "status": "ERROR", "raw_text_len": 0, "tables_count": 0})
    elif extraction:
        summary.append({
            "eleve_id": eleve_id,
            "status": "OK",
            "raw_text_len": len(extraction.raw_text or ""),
            "tables_count": len(extraction.raw_tables or []),
        })
    else:
        summary.append({"eleve_id": eleve_id, "status": "EMPTY", "raw_text_len": 0, "tables_count": 0})

df_summary = pd.DataFrame(summary)
print(df_summary.to_string(index=False))

## 4. Prochaines étapes

- [ ] Pseudonymisation (optionnel, si envoi cloud)
- [ ] Stockage DuckDB
- [ ] Génération LLM des synthèses